# <pre>Time Series Forecasting Dashboard using Streamlit

## Starting ngrok tunnel

In [ ]:
!pip install streamlit
!pip install pyngrok

In [19]:
!streamlit run app.py &>/content/logs.txt &

In [151]:
%%writefile app.py
import numpy as np
import pandas as pd
import tensorflow as tf
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

import streamlit as st
import plotly.express as px
from pyngrok import ngrok

Overwriting app.py


In [16]:
#specifying the autorization token
!ngrok authtoken 2ms1mDrkTaNFMOKnok6Tg13iLZp_5x49SPxvirKskFtXgpeJm

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [17]:
url = ngrok.connect(8501, 'http')
print(f'Access the Time Series Dashboard at {url}')

Access the Time Series Dashboard at NgrokTunnel: "https://7f63-34-138-170-133.ngrok-free.app" -> "http://localhost:8501"


## 7. Writing the Streamlit Code

In [152]:
%%writefile -a app.py

#importing the dataset
amazon_stock_price_df = pd.read_csv('/content/mydata.csv')

#setting page configuration
st.set_page_config(page_title = 'Time Series Dashboard',
                   page_icon =':date:',
                   layout = 'wide')

Appending to app.py


In [153]:
%%writefile -a app.py

#setting page title
st.title('Dashboard for Stock Price Prediction 	:chart:')

#moving page title upwards and in center
st.markdown(
    '''<style>
            div.block-container{
              padding-top:2rem;
              text-align:center;
            }
       </style>
    ''',
    unsafe_allow_html = True)

Appending to app.py


**DATA PREPROCESSING**

In [154]:
%%writefile -a app.py
#including important columns
amazon_stock_price_df = amazon_stock_price_df[['Date', 'Open', 'High', 'Low', 'Close']]
#converting date column to 'datetime' datatype
amazon_stock_price_df['Date'] = pd.to_datetime(amazon_stock_price_df['Date'])
#making the 'date' column as index
amazon_stock_price_df.set_index('Date', inplace = True)

Appending to app.py


**CREATING DATE FILTERS**

In [155]:
%%writefile -a app.py

import pandas as pd
col1, col2 = st.columns((2))

#getting min and max date
startDate = pd.to_datetime(amazon_stock_price_df.index).min()
endDate = pd.to_datetime(amazon_stock_price_df.index).max()

with col1:
  date1 = pd.to_datetime(st.date_input("Start Date", startDate))
with col2:
  date2 = pd.to_datetime(st.date_input("End Date", endDate))

df = amazon_stock_price_df.loc[date1:date2]

Appending to app.py


**DATA NORMALIZATION**

In [156]:
%%writefile -a app.py
#data normalization
scaler = MinMaxScaler()
data = scaler.fit_transform(df[df.columns])

Appending to app.py


**CREATING SLIDING WINDOW SEQUENCES**

In [157]:
%%writefile -a app.py
WINDOW_SIZE = 20
def create_sliding_window(data, window_size):
  X, y = [], []
  for i in range(len(data) - window_size):
    X.append(data[i:i + window_size, :])  # Use all features
    y.append(data[i + window_size, :])    # Predict all columns (Open, High, Low, Close)
  return np.array(X), np.array(y)
X,  y = create_sliding_window(data, WINDOW_SIZE)

Appending to app.py


**BUILDING CNN MODEL**

In [158]:
%%writefile -a app.py
model = tf.keras.Sequential([
    # CONV 1D LAYER
    tf.keras.layers.Conv1D(filters = 64, kernel_size = 2, activation = 'relu', input_shape = (X.shape[1], X.shape[2])),
    # FLATTENING LAYER
    tf.keras.layers.Flatten(),
    # DENSE HIDDEN LAYER
    tf.keras.layers.Dense(50, activation = 'relu'),
    # DENSE OUTPUT LAYER
    tf.keras.layers.Dense(X.shape[2])
])
# COMPILATION
model.compile(optimizer = 'adam',
              loss = 'mse',
              metrics = ['accuracy'])
#TRAINING THE MODEL
model.fit(X, y, epochs = 25, batch_size = 8)

Appending to app.py


**MAKING PREDICTIONS**

In [159]:
%%writefile -a app.py
preds = model.predict(X)

#transforming data to original form
preds = scaler.inverse_transform(preds)

Appending to app.py


**PLOTTING THE DATA**

In [160]:
%%writefile -a app.py

# Plot actual vs predicted prices using Plotly
for idx, col in enumerate(amazon_stock_price_df.columns):
    st.subheader(f'{col}')
    fig = go.Figure()

    # Plot actual data
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df[col],
        mode='lines',
        name=f'Actual {col}'
    ))

    # Plot predicted data
    fig.add_trace(go.Scatter(
        x=df.index[:len(preds)],
        y=preds[:, idx],
        mode='lines',
        name=f'Predicted {col}',
        line=dict(color = 'yellow')
    ))

    st.plotly_chart(fig)

Appending to app.py
